In [1]:
import os
import sys
from pyspark import SparkContext, SparkConf
import json
import itertools
import math

In [169]:
# ta feng
import csv

def extract_csv(file_names):
    for f in file_names:
        with open(f, 'rt') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader):
                if(i == 0):
                    continue
                yield line

rdd = sc.parallelize(['./ta_feng_all_months_merged.csv']).mapPartitions(extract_csv)
header = rdd.first()

rdd = rdd.map(lambda x: '{}-{},{}'.format(x[0], x[1], int(x[5]))).collect()

In [170]:
with open("preprocessed_data.csv", "wt") as f:
    f.write("DATE-CUSTOMER_ID,PRODUCT_ID\n")
    for i, l in enumerate(rdd):    
        f.write(l+"\n")

In [176]:
baskets_rdd = reading_baskets('preprocessed_data.csv', 1, 20)

In [177]:
baskets_rdd.collect()[:2]

[('11/1/2000-01849332',
  ['4710126031890',
   '4710085120628',
   '4710304111147',
   '4710857000011',
   '4711524000433',
   '4714981010038',
   '9310034370095',
   '4710085172696',
   '4713608210912',
   '7610700600863',
   '4711524000907',
   '4901201101021',
   '4710011405133',
   '4710047502011',
   '4710199010778',
   '9310135026372',
   '4711524000891',
   '4710011409056',
   '4710011406123',
   '4903065141502',
   '4719862260427',
   '4710047512515',
   '4710011432832',
   '4711524001041',
   '4710098166811',
   '89782012029',
   '4710126092105',
   '4710126092129']),
 ('11/1/2000-02154039',
  ['4719090900058',
   '4710085120628',
   '4710144101452',
   '4710199010174',
   '4710265849066',
   '4710085120093',
   '4710583110015',
   '20415723',
   '4710247005831',
   '4710105026411',
   '4710088410139',
   '4710085172696',
   '723125481409',
   '4710254031342',
   '4710011401128',
   '4711080010112',
   '4710670200407',
   '4710088414052',
   '4710670200100',
   '4710011405133'

In [171]:
def read_csv_line(line):
    line = line.split(',')
    return (line[0].strip(), line[1].strip())

def reading_baskets(input_file_path, case_number):
    rdd = sc.textFile(input_file_path)
    header = rdd.first()
    
    if(case_number == 1):
        rdd = rdd.filter(lambda x: x != header).map(read_csv_line).groupByKey().map(lambda x : (x[0], list(set(x[1]))))
    else:
        rdd = rdd.filter(lambda x: x != header).map(read_csv_line).map(lambda x: (x[1], x[0])).groupByKey().map(lambda x : (x[0], list(set(x[1]))))
    return rdd

# def gen_candidates(baskets, frequent_items, k):
#     candidates = []
#     if(k == 1):
#         for _, b in baskets:
#             for c in itertools.combinations(b, k):
#                 candidates.append(frozenset(c))
                
#         return list(set(candidates))
    
#     for _, b in baskets:
#         for c in itertools.combinations(b, k):
#             sub_combinations = set(frozenset(x) for x in itertools.combinations(c, k-1))
#             if(sub_combinations.issubset(set(frequent_items))):
#                 candidates.append(frozenset(c))
                
#     return list(set(candidates))

def gen_candidates(baskets, frequent_items, k):
    candidates = []
    if(k == 1):
        for _, b in baskets:
            for c in itertools.combinations(b, k):
                candidates.append(frozenset(c))
                
        return list(set(candidates))
    
#     for _, b in baskets:
#         for c in itertools.combinations(b, k):
    for s1, s2 in itertools.combinations(frequent_items, 2):
        c = s1.union(s2)
        if(len(c) == k):
            candidates.append(frozenset(c))
            
    return list(set(candidates))
    
    
def calculate_support(baskets, candidate):
    count = 0
#     k = len(candidate)
    candidate = set(candidate)
    for _, b in baskets:
        b = set(b)
#         combinations = set(frozenset(x) for x in itertools.combinations(b, k))
        if(candidate.issubset(b)):
            count += 1
    return count

# def calculate_support(baskets, k):
#     counter = {}
#     for _, b in baskets:
# #         b = list(b)
#         combinations = [frozenset(x) for x in itertools.combinations(b, k)]
#         for c in combinations:
#             if(c in counter):
#                 counter[c] += 1
#             else:
#                 counter[c] = 1
#     return counter


def apriori(baskets, sup):
    k = 1
    candidates = gen_candidates(baskets, [], 1)
    output = []
    while(len(candidates) != 0):
        frequent_items = []
        for c in candidates:
            if(calculate_support(baskets, c) >= sup):
                frequent_items.append(c)
        
#         output['candidate'][k] = candidates
        output += frequent_items
        k += 1
        candidates = gen_candidates(baskets, frequent_items, k)
        
    return list(set(output))



def apriori_son(baskets_iterator, sup, total_len):
    baskets = []
    len_partition = 0
    for b in baskets_iterator:
        baskets.append(b)
        len_partition += 1
        
    partition_sup = math.ceil(sup * len_partition / total_len)
#     yield (partition_sup, len_partition, total_len)
    result = apriori(baskets, partition_sup)
    for x in result:
        yield x

    
def phase2_count(basket_iterator, candidates):
    baskets = [x for x in basket_iterator]
    for c in candidates:
        yield (c, calculate_support(baskets, c))
        
        
def prepare_output(candidates, frequent):
    output_candidate = {}
    for item in candidates:
        item = tuple(sorted(tuple(item)))
        if(len(item) in output_candidate):
            output_candidate[len(item)].append(item)
        else:
            output_candidate[len(item)] = [item]
    
    output_frequent = {}
    for item, _ in frequent:
        item = tuple(sorted(tuple(item)))
        if(len(item) in output_frequent):
            output_frequent[len(item)].append(item)
        else:
            output_frequent[len(item)] = [item]
            
    for k, v in output_candidate.items():
        output_candidate[k] = sorted(v)
        
    for k, v in output_frequent.items():
        output_frequent[k] = sorted(v)
        
        
    for k, v in output_candidate.items():
        if(k == 1):
            output_candidate[k] = ",".join(["({})".format(x[0]) for x in v])
            continue
        output_candidate[k] = ",".join([str(x) for x in v])
        
        
    for k, v in output_frequent.items():
        if(k == 1):
            output_frequent[k] = ",".join(["({})".format(x[0]) for x in v])
            continue
        output_frequent[k] = ",".join([str(x) for x in v])
    
            
    return output_candidate, output_frequent


def save_output(output_candidate, output_frequent, output_path):
    with open(output_path, 'wt') as file:
        file.write("Candidates:\n")
        for k in sorted(output_candidate.keys()):
            file.write(output_candidate[k]+'\n')
            file.write('\n')
        file.write("Frequent Itemsets:\n")
        for k in sorted(output_frequent.keys()):
            file.write(output_frequent[k]+'\n')
            file.write('\n')
            


In [3]:
# appName = 'assignment2'
# master = 'local[*]'
# conf = SparkConf().setAppName(appName).setMaster(master)
config = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])
sc = SparkContext(conf=config)
# sc = SparkContext(conf=conf)
sc.setLogLevel("INFO")

In [137]:
baskets_rdd = reading_baskets('small1.csv', 1)

In [138]:
# o = apriori(baskets_rdd.collect(), 9)

In [139]:
# output_candidate = {}
# for item in o:
#     item = tuple(sorted(tuple(item)))
#     if(len(item) in output_candidate):
#         output_candidate[len(item)].append(item)
#     else:
#         output_candidate[len(item)] = [item]
        
# for k, v in output_candidate.items():
#     output_candidate[k] = sorted(v)

# for k, v in output_candidate.items():
#     output_candidate[k] = ",".join([str(x) for x in v])

In [174]:
def reading_baskets(input_file_path, case_number, filter_thresh):
    rdd = sc.textFile(input_file_path)
    header = rdd.first()
    
    if(case_number == 1):
        rdd = rdd.filter(lambda x: x != header).map(read_csv_line).groupByKey().filter(lambda x: len(x[1]) > 20).map(lambda x : (x[0], list(set(x[1]))))
    else:
        rdd = rdd.filter(lambda x: x != header).map(read_csv_line).map(lambda x: (x[1], x[0])).groupByKey().filter(lambda x: len(x[1]) > 20).map(lambda x : (x[0], list(set(x[1]))))
    return rdd

support = 50
filter_threshold = 20
baskets_rdd = reading_baskets("preprocessed_data.csv", 1, filter_threshold)
baskets_rdd = baskets_rdd.repartition(16)
total_len = baskets_rdd.count()
print(total_len, baskets_rdd.getNumPartitions())
phase1 = baskets_rdd.mapPartitions(lambda x: apriori_son(x, support, total_len))
candidates = list(set(phase1.collect()))

5652 16


In [141]:
phase2 = baskets_rdd.mapPartitions(lambda x: phase2_count(x, candidates)).reduceByKey(lambda a, b: a+b).filter(lambda x: x[1] >= support)
frequent = phase2.collect()

In [142]:
o_c, o_f = prepare_output(candidates, frequent)

In [144]:
o_f

{2: "('100', '101'),('100', '98'),('101', '102'),('101', '97'),('101', '98'),('101', '99'),('102', '103'),('102', '97'),('102', '98'),('102', '99'),('103', '99'),('97', '98'),('97', '99'),('98', '99')",
 1: '(100),(101),(102),(103),(97),(98),(99)',
 3: "('100', '101', '98'),('101', '97', '99'),('102', '103', '99'),('97', '98', '99')"}

In [ ]:
save_output(o_c, o_f, 'output2.txt')

In [27]:
a = [frozenset(x) for x in itertools.combinations(range(0, 10), 2)]

In [32]:
len(a[0].union(a[1]))

3

In [111]:
tuple([1])

(1,)